<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=282537033" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=280113860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Library importing

In [1]:
!pip install xgboost
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import gc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 23.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 29.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


# data pre-proccessing

## data reading

In [2]:
import os

for root, dirs, files in os.walk("/kaggle/input", topdown=True):
    print(root)
    for f in files:
        print("   -", f)

/kaggle/input
   - pre-proccess.pkl
   - data.csv


In [3]:
chunks = pd.read_csv("/kaggle/input/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [4]:
del chunks

In [5]:
preprocessor=joblib.load("/kaggle/input/pre-proccess.pkl")

In [6]:
# Define columns
categorical_cols = ["item_category", "item_subcategory", "item_number"]
numeric_cols = ["wm_yr_wk", "wday", "snap", "year", "month", "day", "sell_price", 
                "lag_1", "price_flag", "lag_7", "snap_weekend", "wday_x_snap", 
                "is_weekend", "event_impact", "event_count", "is_event"]

# Define Pipeline
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"

preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])

# --- NEW STEP: FIT THE PREPROCESSOR ---
print("Fitting preprocessor on data...")
# We prepare the X data exactly like we do in the training loop
# We use the whole final_data to ensure it learns ALL categories and ranges
X_fit = final_data.drop(columns=['sales', 'store_id'], errors='ignore') 
preprocessor.fit(X_fit)
print("✅ Preprocessor is now fitted!")
# --------------------------------------

# Now save the FITTED preprocessor
joblib.dump(preprocessor, "/kaggle/working/pre-proccess.pkl")

Fitting preprocessor on data...
✅ Preprocessor is now fitted!


['/kaggle/working/pre-proccess.pkl']

## Data spliting

In [7]:
store_models = {}

stores = final_data["store_id"].unique()

for store in stores:
    print(f"\n==== Training model for store: {store} ====")

    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X.drop(columns='store_id',inplace=True)
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )

    # 4️⃣ Fit model
    model = XGBRegressor(
    objective='reg:tweedie',
    tweedie_variance_power=1.35,
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=15,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=5000,
    early_stopping_rounds=40,
    gamma= 1,
    max_delta_step = 1
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose = False
    )
    
    model.score(X_train,y_train) , model.score(X_test,y_test)
    from sklearn.metrics import mean_absolute_error, r2_score

    train_mae = mean_absolute_error(y_train, model.predict(X_train))
    test_mae  = mean_absolute_error(y_test,  model.predict(X_test))
    train_r2  = r2_score(y_train, model.predict(X_train))
    test_r2   = r2_score(y_test,  model.predict(X_test))

    print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
    print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")
   
    # 5️⃣ Save model
    store_models[store] = model
    print(f"Model for {store} trained and stored.")

    del X, y, X_temp, X_test, X_val, X_train, y_train, y_temp, y_test
    gc.collect()


==== Training model for store: CA_1 ====


/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:05:59] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:05:59] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


MAE train: 0.3920, test: 0.4166
R2  train: 0.9238, test: 0.8834
Model for CA_1 trained and stored.

==== Training model for store: CA_2 ====


/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:10:14] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:10:14] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


MAE train: 0.3156, test: 0.3352
R2  train: 0.8887, test: 0.8454
Model for CA_2 trained and stored.

==== Training model for store: CA_3 ====


...

MAE train: 0.2818, test: 0.2978
R2  train: 0.8950, test: 0.8687
Model for WI_1 trained and stored.

==== Training model for store: WI_2 ====


/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:39:45] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:39:45] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


MAE train: 0.3484, test: 0.3789
R2  train: 0.9202, test: 0.8809
Model for WI_2 trained and stored.

==== Training model for store: WI_3 ====


/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:43:58] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:43:58] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


MAE train: 0.3238, test: 0.3507
R2  train: 0.9367, test: 0.8985
Model for WI_3 trained and stored.


In [8]:
import numpy as np
for store in stores:
    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )
    y_pred = store_models[store].predict(X_test)
    # Round predictions and ground truth
    y_pred_rounded = np.round(y_pred).astype(int)
    y_test_rounded = np.round(y_test).astype(int)

    print("\n=== Sample Predictions vs Actuals ===\n")
    for i in range(min(25, len(y_test))):
        print(f"Pred: {y_pred_rounded[i]:>3}   Actual: {y_test_rounded.iloc[i]:>3}")


=== Sample Predictions vs Actuals ===

Pred:   2   Actual:   3
Pred:   3   Actual:   2
Pred:  11   Actual:   8
Pred:   3   Actual:   3
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   2
Pred:   6   Actual:   6
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   1   Actual:   2
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   3
Pred:   6   Actual:   6
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   2
Pred:   0   Actual:   0
Pred:   0   Actual:   0

=== Sample Predictions vs Actuals ===

Pred:   0   Actual:   0
Pred:   3   Actual:   3
Pred:  11   Actual:   6
Pred:   2   Actual:   0
Pred:   1   Actual:   1
Pred:   2   Actual:   3
Pred:   0   Actual:   0
Pred:   1   Actual:   1
Pred:   1   Actual:   1
Pred:   1   Actual:   1
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   

In [10]:
for i in store_models.keys():
    joblib.dump(store_models[i], f"/kaggle/working/{i}.pkl")